# Random Forest and regression to overcome obstacles

## Import the relevant libraries

In [54]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split

In [55]:
df = pd.read_csv("https://raw.githubusercontent.com/charlesp1996/Pythonproject/main/data.csv")

In [56]:
df

,region_nord,region_centre,region_sud,Month Value,Year Value,Vitesse du vent Axa0 100m (m/s),Production éolienne (GWh),Rayonnement solaire global (W/m2),Production solaire (GWh)
0,1,0,0,2,2014,NaN,6.518965,NaN,2.931509
1,0,0,1,3,2014,NaN,220.644746,NaN,116.568735
2,0,0,1,4,2014,NaN,189.563606,NaN,125.730294
3,0,0,1,4,2014,NaN,9.671714,NaN,98.951709
4,1,0,0,5,2014,NaN,363.412141,NaN,58.899353
...,...,...,...,...,...,...,...,...,...
1248,0,0,1,3,2021,0.0,NaN,5.82,NaN
1249,0,0,1,3,2021,0.0,NaN,6.74,NaN
1250,0,1,0,3,2021,0.0,NaN,7.15,NaN
1251,0,0,1,3,2021,0.0,NaN,6.04,NaN


In [57]:
df = df[df['Rayonnement solaire global (W/m2)'].notna()]

In [58]:
cols = df.columns.tolist()
cols

['region_nord',
 'region_centre',
 'region_sud',
 'Month Value',
 'Year Value',
 'Vitesse du vent Axa0 100m (m/s)',
 'Production éolienne (GWh)',
 'Rayonnement solaire global (W/m2)',
 'Production solaire (GWh)']

### Solaire

In [59]:
colss = cols[:5] + cols[7:9]

In [60]:
colss

['region_nord',
 'region_centre',
 'region_sud',
 'Month Value',
 'Year Value',
 'Rayonnement solaire global (W/m2)',
 'Production solaire (GWh)']

In [61]:
dfs = df[colss]

In [62]:
dfs

,region_nord,region_centre,region_sud,Month Value,Year Value,Rayonnement solaire global (W/m2),Production solaire (GWh)
47,0,1,0,1,2016,7.466964,7.040643
48,1,0,0,1,2016,8.295263,5.596140
49,0,1,0,2,2016,9.242543,23.337292
50,0,1,0,2,2016,9.946336,9.640264
51,0,0,1,2,2016,6.508233,116.805294
...,...,...,...,...,...,...,...
1248,0,0,1,3,2021,5.820000,NaN
1249,0,0,1,3,2021,6.740000,NaN
1250,0,1,0,3,2021,7.150000,NaN
1251,0,0,1,3,2021,6.040000,NaN


In [63]:
X,y = make_regression(n_samples=821, n_features=6,
                                 n_informative=5, n_targets=1,  
                                 tail_strength=0.5,  
                                 shuffle=True, coef=False, random_state=0)
notnans = dfs['Production solaire (GWh)'].notnull()
df_notnans = df[notnans]
X_train, X_test, y_train, y_test = train_test_split(df_notnans[colss[:6]], df_notnans[colss[6:7]],
                                                    train_size=0.75,
                                                    random_state=250)

In [64]:
regr_multirf = MultiOutputRegressor(RandomForestRegressor(max_depth=30,
                                                          random_state=0))

# Fit on the train data
regr_multirf.fit(X_train, y_train)

# Check the prediction score
score = regr_multirf.score(X_test, y_test)
print("The prediction score on the test data is {:.2f}%".format(score*100))

The prediction score on the test data is 64.75%


In [65]:
df_nans = dfs.loc[~notnans].copy()
df_nans[colss[6:7]] = regr_multirf.predict(df_nans[colss[0:6]])
df_nans

,region_nord,region_centre,region_sud,Month Value,Year Value,Rayonnement solaire global (W/m2),Production solaire (GWh)
1092,1,0,0,1,2016,6.130850,10.805883
1093,0,0,0,1,2016,7.882551,7.651051
1094,1,0,0,2,2016,6.926638,7.326095
1095,0,0,0,2,2016,8.113793,12.297672
1096,1,0,0,3,2016,5.711842,16.241297
...,...,...,...,...,...,...,...
1248,0,0,1,3,2021,5.820000,236.481544
1249,0,0,1,3,2021,6.740000,214.822227
1250,0,1,0,3,2021,7.150000,28.393754
1251,0,0,1,3,2021,6.040000,187.794577


# Create the targets

In [66]:
targets = np.where(df_nans['Production solaire (GWh)'] > df_nans['Production solaire (GWh)'].median(), 1, 0)

In [67]:
targets.shape

(161,)

In [68]:
df_nans['Execessive Solar'] = targets

# A comment on the targets

In [69]:
targets.sum() / targets.shape[0]

0.4968944099378882

In [70]:
data_with_targets = df_nans

In [71]:
data_with_targets.columns.values

array(['region_nord', 'region_centre', 'region_sud', 'Month Value',
       'Year Value', 'Rayonnement solaire global (W/m2)',
       'Production solaire (GWh)', 'Execessive Solar'], dtype=object)

In [72]:
data_with_targets = data_with_targets.drop(data_with_targets.columns[6],axis=1)

In [73]:
data_with_targets

,region_nord,region_centre,region_sud,Month Value,Year Value,Rayonnement solaire global (W/m2),Execessive Solar
1092,1,0,0,1,2016,6.130850,0
1093,0,0,0,1,2016,7.882551,0
1094,1,0,0,2,2016,6.926638,0
1095,0,0,0,2,2016,8.113793,0
1096,1,0,0,3,2016,5.711842,0
...,...,...,...,...,...,...,...
1248,0,0,1,3,2021,5.820000,1
1249,0,0,1,3,2021,6.740000,1
1250,0,1,0,3,2021,7.150000,1
1251,0,0,1,3,2021,6.040000,1


# Select the inputs for the regression

In [74]:
data_with_targets.shape

(161, 7)

In [75]:
unscaled_inputs = data_with_targets.iloc[:,:-1]
unscaled_inputs

,region_nord,region_centre,region_sud,Month Value,Year Value,Rayonnement solaire global (W/m2)
1092,1,0,0,1,2016,6.130850
1093,0,0,0,1,2016,7.882551
1094,1,0,0,2,2016,6.926638
1095,0,0,0,2,2016,8.113793
1096,1,0,0,3,2016,5.711842
...,...,...,...,...,...,...
1248,0,0,1,3,2021,5.820000
1249,0,0,1,3,2021,6.740000
1250,0,1,0,3,2021,7.150000
1251,0,0,1,3,2021,6.040000


# Standardize the data

In [76]:
from sklearn.preprocessing import StandardScaler

solar_scaler = StandardScaler(copy=True, with_mean=True, with_std=True)

In [77]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class CustomScaler(BaseEstimator, TransformerMixin):
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        self.scaler = StandardScaler(copy,with_mean,with_std)
        self.columns = columns
        self.mean_ = None
        self.vr_ = None
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        X_not_scaled= X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled,X_scaled], axis=1)[init_col_order]

In [78]:
unscaled_inputs.columns.values

array(['region_nord', 'region_centre', 'region_sud', 'Month Value',
       'Year Value', 'Rayonnement solaire global (W/m2)'], dtype=object)

In [79]:
columns_to_scale = ['region_sud', 'region_nord', 'region_centre', 'Month Value',
       'Year Value', 'Rayonnement solaire global (W/m2)', 'Production solaire (GWh)']
columns_to_omit = []

In [80]:
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [81]:
solar_scaler = CustomScaler(columns_to_scale)

In [82]:
solar_scaler.fit(unscaled_inputs)

C:\Users\charl\anaconda3\lib\site-packages\sklearn\base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CustomScaler(columns=['region_nord', 'region_centre', 'region_sud',
                      'Month Value', 'Year Value',
                      'Rayonnement solaire global (W/m2)'],
             copy=None, with_mean=None, with_std=None)

In [83]:
scaled_inputs = solar_scaler.transform(unscaled_inputs)

In [84]:
solar_scaler.fit(unscaled_inputs)

C:\Users\charl\anaconda3\lib\site-packages\sklearn\base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CustomScaler(columns=['region_nord', 'region_centre', 'region_sud',
                      'Month Value', 'Year Value',
                      'Rayonnement solaire global (W/m2)'],
             copy=None, with_mean=None, with_std=None)

In [85]:
scaled_inputs = solar_scaler.transform(unscaled_inputs)

In [86]:
scaled_inputs[:161][:]

,region_nord,region_centre,region_sud,Month Value,Year Value,Rayonnement solaire global (W/m2)
0,1.097943,-0.283790,-0.320530,-1.204950,-1.545362,0.262034
1,-0.910794,-0.283790,-0.320530,-1.204950,-1.545362,1.672103
2,1.097943,-0.283790,-0.320530,-0.927415,-1.545362,0.902621
3,-0.910794,-0.283790,-0.320530,-0.927415,-1.545362,1.858247
4,1.097943,-0.283790,-0.320530,-0.649880,-1.545362,-0.075255
...,...,...,...,...,...,...
156,-0.910794,-0.283790,3.119829,-0.649880,1.250181,0.011809
157,-0.910794,-0.283790,3.119829,-0.649880,1.250181,0.752383
158,-0.910794,3.523729,-0.320530,-0.649880,1.250181,1.082421
159,-0.910794,-0.283790,3.119829,-0.649880,1.250181,0.188903


In [87]:
scaled_inputs[:161][:].shape

(161, 6)

# Split the data into train & test and shuffle

## Import the relevant module 

In [88]:
from sklearn.model_selection import train_test_split

## Split

In [89]:
train_test_split(scaled_inputs[:161][:], targets)

[     region_nord  region_centre  region_sud  Month Value  Year Value  \
 77     -0.910794       -0.28379   -0.320530    -0.649880    0.131964   
 142     1.097943       -0.28379   -0.320530    -0.927415    1.250181   
 21     -0.910794       -0.28379   -0.320530     1.570400   -1.545362   
 24      1.097943       -0.28379   -0.320530    -1.204950   -0.986254   
 134     1.097943       -0.28379   -0.320530    -1.204950    1.250181   
 ..           ...            ...         ...          ...         ...   
 36      1.097943       -0.28379   -0.320530     0.460260   -0.986254   
 44      1.097943       -0.28379   -0.320530     1.570400   -0.986254   
 148    -0.910794       -0.28379    3.119829    -0.649880    1.250181   
 100     1.097943       -0.28379   -0.320530    -0.649880    0.691072   
 110     1.097943       -0.28379   -0.320530     0.737795    0.691072   
 
      Rayonnement solaire global (W/m2)  
 77                            1.683358  
 142                           1.87646

In [90]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs[:161][:], targets, train_size = 0.8, random_state = 20)

In [91]:
print(x_train.shape, y_train.shape)

(128, 6) (128,)


In [92]:
print(x_test.shape, y_test.shape)

(33, 6) (33,)


# Logisitc regression with sklearn

In [93]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# Training the model

In [94]:
reg = LogisticRegression()

In [95]:
reg.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [96]:
reg.score(x_train, y_train)

0.8671875

### Manually check accuracy

In [97]:
model_outputs = reg.predict(x_train)
model_outputs

array([1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0])

In [98]:
y_train

array([1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0])

In [99]:
model_outputs == y_train

array([ True,  True,  True,  True,  True,  True, False,  True,  True,
       False,  True,  True,  True, False,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True, False, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False,  True,  True, False,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True]

In [100]:
np.sum(model_outputs==y_train)

111

In [101]:
model_outputs.shape[0]

128

In [102]:
np.sum(model_outputs==y_train)/model_outputs.shape[0]

0.8671875

### Finding the intercept and coefficients

In [103]:
reg.intercept_

array([0.07698082])

In [104]:
reg.coef_

array([[-1.57189917e+00,  1.04637311e-03,  5.98290630e-01,
        -2.71429570e-01,  1.37915431e+00, -2.21098704e+00]])

In [105]:
unscaled_inputs.columns.values

array(['region_nord', 'region_centre', 'region_sud', 'Month Value',
       'Year Value', 'Rayonnement solaire global (W/m2)'], dtype=object)

In [106]:
feature_name = unscaled_inputs.columns.values

In [107]:
summary_table = pd.DataFrame (columns=['feature_name'], data = feature_name)
summary_table['Coefficient'] = np.transpose(reg.coef_)
summary_table

,feature_name,Coefficient
0,region_nord,-1.571899
1,region_centre,0.001046
2,region_sud,0.598291
3,Month Value,-0.271430
4,Year Value,1.379154
5,Rayonnement solaire global (W/m2),-2.210987


In [108]:
summary_table.index = summary_table.index+1

In [109]:
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,feature_name,Coefficient
0,Intercept,0.076981
1,region_nord,-1.571899
2,region_centre,0.001046
3,region_sud,0.598291
4,Month Value,-0.271430
5,Year Value,1.379154
6,Rayonnement solaire global (W/m2),-2.210987


## Interpreting the coefficient

In [110]:
summary_table['Odd_ratio'] = np.exp(summary_table.Coefficient)

In [111]:
summary_table

,feature_name,Coefficient,Odd_ratio
0,Intercept,0.076981,1.080021
1,region_nord,-1.571899,0.207650
2,region_centre,0.001046,1.001047
3,region_sud,0.598291,1.819007
4,Month Value,-0.271430,0.762289
5,Year Value,1.379154,3.971542
6,Rayonnement solaire global (W/m2),-2.210987,0.109592


In [112]:
summary_table.sort_values('Odd_ratio', ascending=False)

,feature_name,Coefficient,Odd_ratio
5,Year Value,1.379154,3.971542
3,region_sud,0.598291,1.819007
0,Intercept,0.076981,1.080021
2,region_centre,0.001046,1.001047
4,Month Value,-0.271430,0.762289
1,region_nord,-1.571899,0.207650
6,Rayonnement solaire global (W/m2),-2.210987,0.109592


# Testing the model

In [113]:
reg.score(x_test, y_test)

0.7878787878787878

In [114]:
predicted_proba = reg.predict_proba(x_test)
predicted_proba

array([[0.00818806, 0.99181194],
       [0.78359195, 0.21640805],
       [0.71294838, 0.28705162],
       [0.12934233, 0.87065767],
       [0.70651011, 0.29348989],
       [0.26551032, 0.73448968],
       [0.70477921, 0.29522079],
       [0.55035261, 0.44964739],
       [0.88563379, 0.11436621],
       [0.03180658, 0.96819342],
       [0.53693725, 0.46306275],
       [0.50957811, 0.49042189],
       [0.35864219, 0.64135781],
       [0.02628794, 0.97371206],
       [0.99622299, 0.00377701],
       [0.72602287, 0.27397713],
       [0.00461126, 0.99538874],
       [0.02685218, 0.97314782],
       [0.00565332, 0.99434668],
       [0.66384402, 0.33615598],
       [0.71094318, 0.28905682],
       [0.95676087, 0.04323913],
       [0.91882179, 0.08117821],
       [0.20389728, 0.79610272],
       [0.11819509, 0.88180491],
       [0.13963966, 0.86036034],
       [0.99506039, 0.00493961],
       [0.26570202, 0.73429798],
       [0.89171278, 0.10828722],
       [0.62508078, 0.37491922],
       [0.

In [115]:
predicted_proba.shape

(33, 2)

In [116]:
predicted_proba[:,1]

array([0.99181194, 0.21640805, 0.28705162, 0.87065767, 0.29348989,
       0.73448968, 0.29522079, 0.44964739, 0.11436621, 0.96819342,
       0.46306275, 0.49042189, 0.64135781, 0.97371206, 0.00377701,
       0.27397713, 0.99538874, 0.97314782, 0.99434668, 0.33615598,
       0.28905682, 0.04323913, 0.08117821, 0.79610272, 0.88180491,
       0.86036034, 0.00493961, 0.73429798, 0.10828722, 0.37491922,
       0.16850557, 0.27395976, 0.33742944])

# Saving the model

In [117]:
import pickle

In [118]:
with open('model','wb') as file:
    pickle.dump(reg, file)

In [119]:
with open('scaler','wb') as file:
    pickle.dump(solar_scaler, file)